# Example data analysis

Execute the following command to run the simulation and generate the data files:
```bash
cd XeSim
make && $G4WORKDIR/bin/Linux-g++/XeSim -d template -p ./macros/template/preinit.mac -c ./macros/template/det_setup.mac -f ./macros/template/src_neutron.mac -n 10000
```

In [1]:
import uproot
import awkward as ak
import numpy as np

## Read meta data

In [2]:
file = "../events.root"
rfile = uproot.open(file)

print(f"The file contains {len(rfile.keys())} keys:")
for key in rfile.keys():
    print(f"  {key}")

print(f"\nExecuted with G4VERSION {rfile['G4VERSION_TAG'].member('fTitle')}")
print(f"Total runtime: {rfile['G4RUNTIME'].member('fVal')} seconds")
print(f"Number of events: {rfile['events/nbevents'].member('fVal')}")

print(f"\nUsing the macros:")
for macro in rfile["macros"].keys():
    print(f"--> {rfile['macros'][macro].member('fName')}")
    print(f"    File content hidden for brevity ...")
    #for line in rfile["macros"][macro].member('fTitle').splitlines():
    #    print(f"    {line}")
    #print("")

The file contains 15 keys:
  G4VERSION_TAG;1
  MC_TAG;1
  MCVERSION_TAG;1
  macros;1
  macros/preinit.mac;1
  macros/det_setup.mac;1
  macros/src_neutron.mac;1
  tables;1
  tables/process_subtypes;1
  tables/logical_volumes;1
  tables/physical_volumes;1
  events;1
  events/nbevents;1
  events/events;1
  G4RUNTIME;1

Executed with G4VERSION $Name: geant4-10-06-patch-03 $
Total runtime: 0.679983898 seconds
Number of events: 100

Using the macros:
--> preinit.mac
    File content hidden for brevity ...
--> det_setup.mac
    File content hidden for brevity ...
--> src_neutron.mac
    File content hidden for brevity ...


In [22]:
# Read tables for types and volumes
physical_volumes = {}
for k,v in zip(*rfile["tables/physical_volumes"].arrays(library="np", how=list)):
    physical_volumes = {**physical_volumes, **{k: v for k, v in zip(k, v)}}

print(f"\nPhysical volumes: {physical_volumes.keys()}")

subtypes_table = rfile["tables/process_subtypes"].arrays(library="np")

particle_ids = {}
for k,v in zip(subtypes_table['particle_id'], subtypes_table['particle_name']):
    particle_ids = {**particle_ids, **{k: v for k, v in zip(k, v)}}

def get_particle_name(particle_id):
    """Get the particle name from the particle PDG code."""
    if particle_id in particle_ids:
        return particle_ids[particle_id]
    else:
        return f"Unknown particle code {particle_id}"

print(f"Particle PDG code of 2112 is {get_particle_name(2112)}.")

process_subtype = {}
for k,v in zip(subtypes_table['process_subtype'], subtypes_table['process_name']):
    process_subtype = {**process_subtype, **{k: v for k, v in zip(k, v)}}

def get_process_name(process_id):
    """Get the process name from the process ID."""
    if process_id in process_subtype:
        return process_subtype[process_id]
    else:
        return f"Unknown process ID {process_id}"

print(f"Process type of 210 is {get_process_name(210)}.")


Physical volumes: dict_keys(['Lab', 'LXe', 'GXe', 'LXePTFECylinder', 'GXePTFECylinder', 'GXePTFETop', 'PmtPhotoCathode', 'LXePTFEBottom'])
Particle PDG code of 2112 is neutron.
Process type of 210 is RadioactiveDecayBase.


## Load small data files